Data source:
- https://www.insee.fr/fr/statistiques/3568638?sommaire=3568656
- https://public.opendatasoft.com/explore/dataset/buildingref-france-bpe-all-geolocated/export/?disjunctive.geocode_quality&disjunctive.equipment_name&disjunctive.category&disjunctive.reg_name&disjunctive.dep_name&disjunctive.epci_name&disjunctive.com_arm_name&disjunctive.com_arm_area_code&sort=year&refine.equipment_name=Supermarch%C3%A9&refine.equipment_name=Sup%C3%A9rette&refine.equipment_name=Hypermarch%C3%A9&location=12,48.84732,2.36361&basemap=jawg.streets

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import configparser
from urllib.parse import urlencode
import requests
import re

from shapely.geometry import Point, shape 
import geopandas as gpd
import folium
import branca.colormap as cm

In [ ]:
bpe19_df = pd.read_csv('buildingref-france-bpe-all-geolocated.csv', sep=';')
bpe19_df = bpe19_df[(bpe19_df['Code Officiel Département'].isin(['974', '971', '976', '972', '973']) == False) &
                    (bpe19_df['Qualité du géocodage'] != 'Non Géolocalisée')]

In [ ]:
bpe19_df['geo_point_list'] = bpe19_df['Geo Point'].apply(lambda x: x.split(','))
bpe19_df['LAT'] = bpe19_df['geo_point_list'].apply(lambda x: x[0])
bpe19_df['LAT'] = bpe19_df['LAT'].astype(float)
bpe19_df['LNG'] = bpe19_df['geo_point_list'].apply(lambda x: x[1])
bpe19_df['LNG'] = bpe19_df['LNG'].astype(float)


In [ ]:
bpe_geometry = [Point(xy) for xy in zip(bpe19_df.LNG, bpe19_df.LAT)]
crs = {'init': 'epsg:4326'}
bpe19_gdf = gpd.GeoDataFrame(bpe19_df, crs=crs, geometry=bpe_geometry)

In [ ]:
bpe19_gdf.head()

In [ ]:
dept_geo = gpd.read_file('../geo_datasets/fr_departements.geojson', driver='GeoJSON')
commune_geo = gpd.read_file('../geo_datasets/fr_contours_communes_2019.geojson', driver='GeoJSON')